<a href="https://colab.research.google.com/github/interritus141/COMP0031-Group-Research-Project/blob/master/COMP0031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

## Installation

## General

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data

In [19]:
from ta import add_all_ta_features
import yfinance as yf

## Environment

In [20]:
import gym 
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, DDPG, DQN, PPO

## Backtesting

In [21]:
from backtesting import Backtest, Strategy

# Data

## Tech Indicators

In [22]:
def add_ta(df):
  ta_df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume")
  # print(ta_df.columns)
  ta_df = ta_df[["Open", "High", "Low", "Close", "Volume", "Adj Close", "volume_obv",
                                "volume_adi", "trend_adx", "momentum_ao", "trend_macd", "momentum_rsi", 
                                "momentum_stoch"]]
  ta_df = ta_df.fillna(ta_df.mean())
  return ta_df

## Stocks

1. Apple Inc. (AAPL)
2. Microsoft Corp. (MSFT)
3. Amazon.com, Inc. ( AMZN)
4. Tesla, Inc. (TSLA)
5. Nvidia Corp. (NVDA)

In [23]:
stock_volatilities = {}
stocks = dict.fromkeys(["AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "CAAS"])

for stock in stocks.keys():
  stock_df = yf.download(stock, start="2018-01-01", end="2022-12-31", keepna=True)
  stock_df = stock_df.fillna(stock_df.mean())
  stock_df = add_ta(stock_df)
  stocks[stock] = stock_df
  stock_volatilities[stock] = None



[*********************100%***********************]  1 of 1 completed


/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


[*********************100%***********************]  1 of 1 completed


/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


[*********************100%***********************]  1 of 1 completed


...

/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


[*********************100%***********************]  1 of 1 completed


/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


[*********************100%***********************]  1 of 1 completed


/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/nono/miniconda3/envs/rl_tutorial/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [24]:
def add_volatility(df, df_name):
  df["Log returns"] = np.log(df['Close'] / df['Close'].shift())
  stock_volatilities[df_name] = df['Log returns'].std() * 252 ** .5


In [25]:
def visualise_volatility(df, df_name, volatility):
  fig, ax = plt.subplots()
  df['Log returns'].hist(ax=ax, bins=50, alpha=0.6, color='b')
  ax.set_xlabel("Log return")
  ax.set_ylabel("Freq of log return")
  ax.set_title("{:s} volatility: {:.2f}%".format(df_name, volatility*100))

for stock, stock_df in stocks.items():
  add_volatility(stock_df, stock)
  # visualise_volatility(stock_df, stock, stock_volatilities[stock]) // use to generate graphs

print(stock_volatilities)

{'AAPL': 0.33515159660010024, 'MSFT': 0.3108167320339789, 'AMZN': 0.35823866187984743, 'TSLA': 0.6545095345365393, 'NVDA': 0.5216282953491833, 'CAAS': 0.83977708799141}


In [26]:
def add_cov(df):
  df = df.reset_index()

  cov_list = []
  return_list = []

  # look back is one year
  lookback=252
  for i in range(lookback,len(df.index.unique())):
    data_lookback = df.iloc[i-lookback:i,:]
    price_lookback=data_lookback.pivot_table(index = 'Date', values = 'Close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values 
    cov_list.append(covs)


  df_cov = pd.DataFrame({'Date':df["Date"].unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
  df = df.merge(df_cov, on='Date')
  df = df.sort_values(['Date']).reset_index(drop=True)
  return df

In [27]:
aapl_df = stocks["AAPL"]
caas_df = stocks["CAAS"]

aapl_df = add_cov(aapl_df)
caas_df = add_cov(caas_df)

In [81]:
new_aapl_df = aapl_df.copy()
new_aapl_df["tic"] = "AAPL"
new_caas_df = caas_df.copy()
new_caas_df["tic"] = "CAAS"
mixed_df = pd.concat([new_aapl_df, new_caas_df])

# Environment

In [29]:
class StockPortfolioEnv(gym.Env):
  def __init__(self):
    pass
  
  def reset(self):
    pass
  
  def step(self, action):
    pass

## Kaggle Tutorial

In [30]:
class StockPortfolioEnv(gym.Env):
  def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
    #super(StockEnv, self).__init__()
    #money = 10 , scope = 1
    self.day = day
    self.lookback=lookback
    self.df = df
    self.stock_dim = stock_dim
    self.hmax = hmax
    self.initial_amount = initial_amount
    self.transaction_cost_pct =transaction_cost_pct
    self.reward_scaling = reward_scaling
    self.state_space = state_space
    self.action_space = action_space
    self.tech_indicator_list = tech_indicator_list

    # action_space normalization and shape is self.stock_dim
    self.action_space = gym.spaces.Box(low = -1, high = 1,shape = (self.action_space,)) 
    # Shape = ??
    # covariance matrix + technical indicators
    self.observation_space = gym.spaces.Box(low=0, high=np.inf, shape = (1+len(self.tech_indicator_list),self.state_space))

    # load data from a pandas dataframe
    self.data = self.df.loc[self.day,:]
    self.covs = [[x[0][0] for x in self.data['cov_list']]]
    self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
    self.terminal = False     
    self.turbulence_threshold = turbulence_threshold        
    # initalize state: inital portfolio return + individual stock return + individual weights
    self.portfolio_value = self.initial_amount

    # memorize portfolio value each step
    self.asset_memory = [self.initial_amount]
    # memorize portfolio return each step
    self.portfolio_return_memory = [0]
    self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
    self.date_memory=[self.data["Date"].unique()[0]]


        
  def step(self, actions):
    # print(self.day)
    self.terminal = self.day >= len(self.df.index.unique())/self.stock_dim-1
    # print(actions)

    if self.terminal:
      df = pd.DataFrame(self.portfolio_return_memory)
      df.columns = ['daily_return']
      plt.plot(df.daily_return.cumsum(),'r')
      # plt.savefig('results/cumulative_reward.png')
      plt.close()

      plt.plot(self.portfolio_return_memory,'r')
      # plt.savefig('results/rewards.png')
      plt.close()

      print("=================================")
      print("begin_total_asset:{}".format(self.asset_memory[0]))           
      print("end_total_asset:{}".format(self.portfolio_value))

      df_daily_return = pd.DataFrame(self.portfolio_return_memory)
      df_daily_return.columns = ['daily_return']
      if df_daily_return['daily_return'].std() !=0:
        sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                  df_daily_return['daily_return'].std()
        print("Sharpe: ",sharpe)
      print("=================================")

      return self.state, self.reward, self.terminal,{}

    else:
      #print("Model actions: ",actions)
      # actions are the portfolio weight
      # normalize to sum of 1
      #if (np.array(actions) - np.array(actions).min()).sum() != 0:
      #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
      #else:
      #  norm_actions = actions
      weights = self.softmax_normalization(actions) 
      #print("Normalized actions: ", weights)
      self.actions_memory.append(weights)
      last_day_memory = self.data

      #load next state
      self.day += 1
      self.data = self.df.loc[self.day,:]
      self.covs = [[x[0][0] for x in self.data['cov_list']]]
      self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
      #print(self.state)
      # calcualte portfolio return
      # individual stocks' return * weight
      portfolio_return = sum(((self.data["Close"].values / last_day_memory["Close"].values)-1)*weights)
      # update portfolio value
      new_portfolio_value = self.portfolio_value*(1+portfolio_return)
      self.portfolio_value = new_portfolio_value

      # save into memory
      self.portfolio_return_memory.append(portfolio_return)
      self.date_memory.append(self.data["Date"].unique()[0])            
      self.asset_memory.append(new_portfolio_value)

      # the reward is the new portfolio value or end portfolo value
      self.reward = new_portfolio_value 
      #print("Step reward: ", self.reward)
      #self.reward = self.reward*self.reward_scaling

    return self.state, self.reward, self.terminal, {}

  def reset(self):
    self.asset_memory = [self.initial_amount]
    self.day = 0
    self.data = self.df.loc[self.day,:]
    # load states
    self.covs = [[x[0][0] for x in self.data['cov_list']]]
    self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
    self.portfolio_value = self.initial_amount
    #self.cost = 0
    #self.trades = 0
    self.terminal = False 
    self.portfolio_return_memory = [0]
    self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
    self.date_memory=[self.data["Date"].unique()[0]] 
    return self.state
    
  def render(self, mode='human'):
    return self.state
      
  def softmax_normalization(self, actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output

  
  def save_asset_memory(self):
    date_list = self.date_memory
    portfolio_return = self.portfolio_return_memory
    #print(len(date_list))
    #print(len(asset_list))
    df_account_value = pd.DataFrame({'Date':date_list,'daily_return':portfolio_return})
    return df_account_value

  def save_action_memory(self):
    # date and close price length must match actions length
    date_list = self.date_memory
    df_date = pd.DataFrame(date_list)
    df_date.columns = ['Date']
    
    action_list = self.actions_memory
    df_actions = pd.DataFrame(action_list)
    df_actions.columns = self.data["tic"]
    df_actions.index = df_date["Date"]
    #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
    return df_actions

  def _seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def get_sb_env(self):
    e = DummyVecEnv([lambda: self])
    obs = e.reset()
    return e, obs

In [31]:
stock_dimension = len(mixed_df["tic"].unique())
state_space = 2

env_kwargs = {
  "hmax": 100, 
  "initial_amount": 1000000, 
  "transaction_cost_pct": 0.001, 
  "state_space": state_space, 
  "stock_dim": stock_dimension, 
  "tech_indicator_list": [
    "volume_obv",
    "volume_adi", 
    "trend_adx", 
    "momentum_ao", 
    "trend_macd", 
    "momentum_rsi", 
    "momentum_stoch"
  ], 
  "action_space": stock_dimension, 
  "reward_scaling": 1e-4
    
}

env = StockPortfolioEnv(df = mixed_df, **env_kwargs)

In [32]:
env_train, _ = env.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Train

In [33]:
policy = "MlpPolicy"

In [34]:
a2c_agent = A2C(policy, env)
ddpg_agent = DDPG(policy, env)
# dqn_agent = DQN(policy, env)
ppo_agent = PPO(policy, env)

In [35]:
print("A2C")
a2c_result = a2c_agent.learn(
    total_timesteps=1000,
)

print("DDPG")
ddpg_result = ddpg_agent.learn(
    total_timesteps=1000,
)

print("PPO")
ppo_result = ppo_agent.learn(
    total_timesteps=1000,
)

A2C
begin_total_asset:1000000
end_total_asset:3788645.746040318
Sharpe:  1.1594215385613917
DDPG
begin_total_asset:1000000
end_total_asset:3958200.771376303
Sharpe:  1.1741405167007712
begin_total_asset:1000000
end_total_asset:3593772.0251140026
Sharpe:  1.1010476644990632
PPO
begin_total_asset:1000000
end_total_asset:4489271.085829534
Sharpe:  1.2756690142609397
begin_total_asset:1000000
end_total_asset:3754974.2745920937
Sharpe:  1.1327959936549432
begin_total_asset:1000000
end_total_asset:3492902.867525758
Sharpe:  1.3198006781019747
begin_total_asset:1000000
end_total_asset:4008068.9731645295
Sharpe:  0.996054233693926


# Predict

In [36]:
a2c_vec_env = a2c_result.get_env()
obs = a2c_vec_env.reset()
for i in range(1000):
    action, _states = a2c_result.predict(obs, deterministic=True)
    obs, reward, done, info = a2c_vec_env.step(action)
    # print("Step", i, reward)
    a2c_vec_env.render()

begin_total_asset:1000000
end_total_asset:4105175.063453941
Sharpe:  1.5037436911883024


# Backtest

## To explore

In [37]:
class MLTrainOnceStrategy(Strategy):
    price_delta = .004  # 0.4%

    def init(self):        
        # agent
        self.agent = ppo_result
        self.vec_env = ppo_result.get_env()
        self.obs = self.vec_env.reset()
        
        # data
        # self.data = mixed_df
        # self.data.set_index("Date")

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')
    
    def next(self):
        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        # current_time = self.data.index[-1]

        # Forecast the next movement
        action, _states = self.agent.predict(self.obs, deterministic=True)
        self.obs, forecast, done, info = self.vec_env.step(action)
        self.vec_env.render()

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)

        if forecast > 1000000 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast <= 1000000 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open 
        # for more than two days
        # for trade in self.trades:
        #     if current_time - trade.entry_time > pd.Timedelta('2 days'):
        #         if trade.is_long:
        #             trade.sl = max(trade.sl, low)
        #         else:
        #             trade.sl = min(trade.sl, high)

In [38]:
bt = Backtest(aapl_df, MLTrainOnceStrategy, commission=.0002, margin=.05)
bt.run()

/tmp/ipykernel_3082/3738323711.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(aapl_df, MLTrainOnceStrategy, commission=.0002, margin=.05)


begin_total_asset:1000000
end_total_asset:4069555.3339893026
Sharpe:  1.2038728120389555


Start                                     0.0
End                                    1006.0
Duration                               1006.0
Exposure Time [%]                   62.264151
Equity Final [$]                    32.985181
Equity Peak [$]                       10000.0
Return [%]                         -99.670148
Buy & Hold Return [%]              265.510909
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.675341
Avg. Drawdown [%]                  -99.675341
Max. Drawdown Duration                 1005.0
Avg. Drawdown Duration                 1005.0
# Trades                                626.0
Win Rate [%]                        11.022364
Best Trade [%]                       0.698976
Worst Trade [%]                     -3.483065
Avg. Trade [%]                    

## BenchMark
After the strategies have been construct, we are going to use buy and hold and constance_rebalancing to compare the efficiency of the algorithm.
We are using DRR, CRR, Varience, and Sharpe ratio to measure the performance of the benchmark algo.

### DRR
The Daily rate of return can be computed by

$$ DRR = { Price_{today} - Price_{prevday}} $$

In [43]:
# use DRR column
def portfolio_daily_rate_of_return(portfolio_df, initial_amount):
    if not ('DRR' in portfolio_df):
        portfolio_df['DRR'] = np.nan
    portfolio_df['DRR'] = ( portfolio_df['Sum'] - np.roll(portfolio_df['Sum'], shift=1))/portfolio_df['Sum']
    portfolio_df.loc[portfolio_df.index[0], 'DRR']= (portfolio_df.loc[portfolio_df.index[0], 'Sum'] -initial_amount) /initial_amount
    return portfolio_df 


### CRR
The Cumulative rate or the sum of the DRR can be computed by

$$ CRR = { Price_{today} - Price_{init}} $$

In [44]:
# use CRR column
def portfolio_cumulative_rate_of_return(portfolio_df, initial_amount):
    if not ('DRR' in portfolio_df):
        portfolio_df = portfolio_daily_rate_of_return(portfolio_df, initial_amount)
    if not ('CRR' in portfolio_df):
        portfolio_df['CRR'] = np.nan
    
    portfolio_df['CRR'] = ( portfolio_df['Sum'] - initial_amount)/initial_amount

    return portfolio_df 

### Varience
The Varience can be computed by using the following equation

$$ Var = { ( DRR - E_{DRR} )^2 \over Time} $$

In [45]:
def portfolio_varience(portfolio_df):
    drr_sum = portfolio_df['DRR'].sum()/portfolio_df.__len__()

    var = np.square(portfolio_df['DRR'] - drr_sum).sum()/portfolio_df.__len__()
    return var

### Sharpe ratio
The ratio can be computed by using the following equation

$$ Sharpe = { R_{portfolio} - R_{riskfree} \over \sigma_{portfolio}} $$

In [46]:
# compute the portfolio sharpe ratio
def portfolio_sharpe_ratio(portfolio_df, port_sd, riskfree=0.0151, at_index=-1):
    asset_return = portfolio_df.loc[portfolio_df.index[at_index], 'CRR']
    return (asset_return - riskfree)/port_sd

### Define a class for running rebalancing algo

In [86]:
class ConstanceBalancingOnTime:
    def __init__(self, 
                df,
                holding_ratio,
                rebalance_time,
                initial_amount,
                transaction_cost_pct,
                lookback=252,
                day = 0):
        self.df = df
        self.rebalance_time = rebalance_time
        self.holding_ratio = np.array(holding_ratio)
        self.initial_amount = initial_amount
        self.transaction_cost_pct = transaction_cost_pct
        self.lookback = lookback
        self.day = day

        self.ticker_list = df["tic"].unique()
        self.stock_count = len(self.ticker_list)
        # the first one is cash on hand followed by the stock in the tickers
        self.asset_amount = self.holding_ratio * initial_amount

        self.portfolio_memory = self._create_initial_memory()
        self._write_memory_at_index(0)

        #for keeping matric
        self.metric = {}

        

    def step(self):
        # incase 
        self.day += 1
        # if (self.day == 1):
        #     current_value = self.initial_amount / (self.transaction_cost_pct * self.holding_ratio[1:].sum() + self.initial_amount)
        #     self.asset_amount = current_value * self.holding_ratio
        #     return
        
        for index in range(self.stock_count):
            try:
                self.asset_amount[index+1] *= self._update_assetprice_by_ratio(int(self.day), self.ticker_list[index])
            except:
                print("error at day",self.day, index)
        
        # rebalancing
        if (self.day % self.rebalance_time == 0):

            after_asset_amount = np.array([])
            
            current_asset_value = self.asset_amount.sum()
            current_asset_ratio = self.asset_amount/current_asset_value

            #check the sign of asset allowcation
            asset_adapt_sign = self.holding_ratio >= current_asset_ratio
                
            asset_matrix = np.array([np.append(self.holding_ratio.copy(), 0)])
            
            for i in range(self.stock_count+1): 
                # the amount of add in that we will add into the asset line
                add_in_asset_line = 1
                if ((self.holding_ratio == current_asset_ratio)[i] ):
                    pass
                elif (asset_adapt_sign[i] ):
                    add_in_asset_line += self.transaction_cost_pct
                else:
                    add_in_asset_line -= self.transaction_cost_pct
                # print(add_in_asset_line)
                current_asset_line = np.zeros(self.stock_count+2)
                current_asset_line[i] = add_in_asset_line
                current_asset_line[-1] = 1
                # print(asset_matrix.shape, current_asset_line.shape)
                # print(asset_matrix, current_asset_line)
                asset_matrix = np.append(asset_matrix.copy(), np.array([current_asset_line.copy()]), axis=0)
            asset_matrix = asset_matrix.T
            
            modify_amount = np.linalg.solve(asset_matrix, np.append(self.asset_amount, 0))[1:]
            self.asset_amount = self.asset_amount - modify_amount
        
        # write the allocation to the memory df
        self._write_memory_at_index(self.day)

    def run(self):
        time_range = self.df.__len__() / self.stock_count
        for i in range(int(time_range)-1):
            self.step()

    def reset(self):
        self.day = 0
        self.asset_amount = self.holding_ratio * self.initial_amount

        self.portfolio_memory = self._create_initial_memory()
        self._write_memory_at_index(0)

    def get_asset_amount(self):
        return self.asset_amount.sum()
    
    def add_return(self, method):
        self.portfolio_memory = method(self.portfolio_memory, self.initial_amount)

    def add_metric(self, new_matric, new_matric_name, **kwargs):
        matric_res = new_matric(self.portfolio_memory, **kwargs)
        self.metric.update({new_matric_name: matric_res}) 

    def _create_initial_memory(self):
        col = np.append(np.array(['Cash']), self.ticker_list)
        col = np.append(col, np.array(['Sum']))
        initial_memory = pd.DataFrame(index=self.df.index.unique(), columns=col)
        
        return initial_memory
    
    def _update_assetprice_by_ratio(self, time_index: int, ticker: str):
        return self.df[self.df['tic'] == ticker].iloc[time_index]['Close']/self.df[self.df['tic'] == ticker].iloc[time_index-1]['Close']
    
    def _write_memory_at_index(self, index: int):
        self.portfolio_memory.iloc[index]['Cash'] = self.asset_amount[0]
        
        for stock_index in range(self.stock_count):
            current_ticker = self.ticker_list[stock_index]
            self.portfolio_memory.iloc[index][current_ticker] = self.asset_amount[stock_index+1]
        self.portfolio_memory.iloc[index]['Sum'] = self.asset_amount.sum()
    


In [87]:
mixed_df["tic"].unique()

array(['AAPL', 'CAAS'], dtype=object)

In [88]:
#df for testing
test_df = mixed_df.copy().set_index('Date')
risk_free_2022 = 0.024

#### Constance rebalance

In [89]:
env_kwargs = {
  "holding_ratio": [0,0.5, 0.5], 
  "rebalance_time": 60,
  "initial_amount": 1000000, 
  "transaction_cost_pct": 0.001
}

cr = ConstanceBalancingOnTime(df = test_df, **env_kwargs)


In [90]:
cr.reset()
cr.run()

In [91]:
cr.add_return(portfolio_daily_rate_of_return)
cr.add_return(portfolio_cumulative_rate_of_return)

cr.add_metric(portfolio_varience, "varience")
env_kwargs = {
  "port_sd": np.sqrt(cr.metric['varience']), 
  "riskfree": risk_free_2022,
}

cr.add_metric(portfolio_sharpe_ratio, "sharpe", **env_kwargs)


In [92]:
print("rebalance time", cr.rebalance_time)
print("initial amount", cr.initial_amount)
print("tickers", cr.ticker_list)
print("stock count", cr.stock_count)
cr.metric

rebalance time 60
initial amount 1000000
tickers ['AAPL' 'CAAS']
stock count 2


{'varience': 0.000997940224303927, 'sharpe': 81.78974044910075}

### Buy and Hold

In [93]:
## let the time to be very high so the portfolio will not rebalance
env_kwargs = {
  "holding_ratio": [0,0.5,0.5], 
  "rebalance_time": 9999999999,
  "initial_amount": 1000000, 
  "transaction_cost_pct": 0.001
}

buyhold = ConstanceBalancingOnTime(df = test_df, **env_kwargs)


In [94]:
buyhold.reset()
buyhold.run()

In [95]:
buyhold.add_return(portfolio_daily_rate_of_return)
buyhold.add_return(portfolio_cumulative_rate_of_return)
buyhold.add_metric(portfolio_varience, "varience")
env_kwargs = {
  "port_sd": np.sqrt(cr.metric['varience']), 
  "riskfree": risk_free_2022,
}

buyhold.add_metric(portfolio_sharpe_ratio, "sharpe", **env_kwargs)

In [96]:
print("rebalance time", buyhold.rebalance_time)
print("initial amount", buyhold.initial_amount)
print("tickers", buyhold.ticker_list)
print("stock count", buyhold.stock_count)


rebalance time 9999999999
initial amount 1000000
tickers ['AAPL' 'CAAS']
stock count 2


In [97]:
buyhold.metric

{'varience': 0.0006578505069293057, 'sharpe': 62.1570957940343}